<a href="https://colab.research.google.com/github/Muzhi1920/awesome-models/blob/main/04-feature_column/02_HashedCategoricalColumn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.ops import string_ops,sparse_ops,array_ops
from tensorflow.python.framework import sparse_tensor as sparse_tensor_lib
from tensorflow.python.feature_column.feature_column_v2 import _to_sparse_input_and_drop_ignore_values
import tensorflow as tf

# 构造SparseTensor
Sparse特征，存储为SparseTensor。dense_shape保证batch_size固定，实际长度以当前batch为准。

In [ ]:
sparse_tensors=sparse_tensor_lib.SparseTensor(indices=[[0,1],[0,2],[2,0],[2,1],[4,0]],values=[1919,2020,2121,3232,1818],dense_shape=[5,3])
sparse_tensors.indices,sparse_tensors.values,sparse_tensors.dense_shape,tf.sparse.to_dense(sparse_tensors,-1)

(<tf.Tensor: shape=(5, 2), dtype=int64, numpy=
 array([[0, 1],
        [0, 2],
        [2, 0],
        [2, 1],
        [4, 0]])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([1919, 2020, 2121, 3232, 1818], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int64, numpy=array([5, 3])>,
 <tf.Tensor: shape=(5, 3), dtype=int32, numpy=
 array([[  -1, 1919, 2020],
        [  -1,   -1,   -1],
        [2121, 3232,   -1],
        [  -1,   -1,   -1],
        [1818,   -1,   -1]], dtype=int32)>)

# HashCategoryColumn

## 应用于LinearModel，Wide&Deep的wide部分实现
sparse_combiner默认为'sum'，用以实现y = sigmoid(w \*x + b)

### wide部分处理离散特征

\_create_weighted_sum

离散特征对应的weights作为lookup_table的embedding_weights，查找到，即为w\*x;
$$y_d = \sum _{i}^{F_d} w_i * x_i$$

#### 获取sparse_tensor

In [ ]:
# sparse_tensors = column.get_sparse_tensors(transformation_cache,state_manager)
# 对id_tensor拍平
# id_tensor = sparse_ops.sparse_reshape(sparse_tensors.id_tensor,
#                                       [array_ops.shape(sparse_tensors.id_tensor)[0], -1])
id_tensor = sparse_ops.sparse_reshape(sparse_tensors,
                                      [array_ops.shape(sparse_tensors)[0], -1])
id_tensor,tf.sparse.to_dense(id_tensor)

(<tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7ff130011150>,
 <tf.Tensor: shape=(5, 3), dtype=int32, numpy=
 array([[   0, 1919, 2020],
        [   0,    0,    0],
        [2121, 3232,    0],
        [   0,    0,    0],
        [1818,    0,    0]], dtype=int32)>)

#### 对weight_tensor拍平

In [ ]:
# weight_tensor = sparse_tensors.weight_tensor
# if weight_tensor is not None:
#     weight_tensor = sparse_ops.sparse_reshape(weight_tensor, [array_ops.shape(weight_tensor)[0], -1])

#### 按照id_tensor 查找embedding

In [ ]:
# return embedding_ops.safe_embedding_lookup_sparse(
#       weight_var,
#       id_tensor,
#       sparse_weights=weight_tensor,
#       combiner=sparse_combiner,
#       name='weighted_sum')

### wide部分处理dense特征

稠密特征对应的weights需创建，调用get_dense_tensor相乘即可。
$$y_s = \sum _{i}^{F_s} w_i * x_i$$

#### 对feature_column获取dense_tensor

In [ ]:
# tensor = column.get_dense_tensor(transformation_cache, state_manager)

#### 获取dense特征的shape=(1,)，batch_size

In [ ]:
# num_elements = column.variable_shape.num_elements()
# batch_size = array_ops.shape(tensor)[0]

#### 计算reshape和matmul

In [ ]:
# tensor = array_ops.reshape(tensor, shape=(batch_size, num_elements))
# return math_ops.matmul(tensor, weight_var, name='weighted_sum')

#### 结论
1. 对离散特征进行embeddings，lookup，实际查询的对应的weights，而非emb；
2. 对dense特征乘以对应weights即可。
3. 再weight_sum即可
$$y = y_d + y_s$$

## 应用于LinearModel，Wide&Deep的deep部分实现

### get_sparse_tensors
get_sparse_tensors 返回 'IdWeightPair'：('id_tensor', 'weight_tensor')
1. id_tensor是维度为[batch_size, bkts]的SparseTensor；
2. weight_tensor是同型的SparseTensor；若`None`则默认权重为1

In [ ]:
sparse_tensors=sparse_tensor_lib.SparseTensor(indices=[[0,1],[0,2],[2,0],[2,1],[4,0]],values=[1919,2020,2121,3232,1818],dense_shape=[5,3])
sparse_tensors.indices,sparse_tensors.values,sparse_tensors.dense_shape,tf.sparse.to_dense(sparse_tensors,-1)

(<tf.Tensor: shape=(5, 2), dtype=int64, numpy=
 array([[0, 1],
        [0, 2],
        [2, 0],
        [2, 1],
        [4, 0]])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([1919, 2020, 2121, 3232, 1818], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int64, numpy=array([5, 3])>,
 <tf.Tensor: shape=(5, 3), dtype=int32, numpy=
 array([[  -1, 1919, 2020],
        [  -1,   -1,   -1],
        [2121, 3232,   -1],
        [  -1,   -1,   -1],
        [1818,   -1,   -1]], dtype=int32)>)

### transform_feature

In [ ]:
# 如果是SparseTensor，直接返回，不进行操作。
aa = _to_sparse_input_and_drop_ignore_values(sparse_tensors)
aa.indices,aa.values,aa.dense_shape

(<tf.Tensor: shape=(5, 2), dtype=int64, numpy=
 array([[0, 1],
        [0, 2],
        [2, 0],
        [2, 1],
        [4, 0]])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([1919, 2020, 2121, 3232, 1818], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int64, numpy=array([5, 3])>)

1. 如果是Tensor，int型空值由`-1`替代，str型由''代替；转SparseTensor后，置空，但不改变batch_size;
2. str直接hash；int需转str再hash。

In [ ]:
# Hash(str)
hash_values = string_ops.string_to_hash_bucket_fast(string_ops.as_string(aa.values), 12, name='hash_sparse')
hash_values

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([6, 9, 0, 4, 9])>

In [ ]:
# 重新构造该SparseTensor，相当于raw sp只是替换了values，返回！
aaa = sparse_tensor_lib.SparseTensor(aa.indices, hash_values, aa.dense_shape)
aaa.indices,aaa.values,aaa.dense_shape

(<tf.Tensor: shape=(5, 2), dtype=int64, numpy=
 array([[0, 1],
        [0, 2],
        [2, 0],
        [2, 1],
        [4, 0]])>,
 <tf.Tensor: shape=(5,), dtype=int64, numpy=array([6, 9, 0, 4, 9])>,
 <tf.Tensor: shape=(2,), dtype=int64, numpy=array([5, 3])>)

### EmbeddingColumn
按照bkts构建embedding_weights，然后按照hash后的int64进行embedding_lookup

# HashCategoryColumn总结
1. 对所有Tensor类型，转SparseTensor；
2. 对str或int进行hash；
3. 构造相同SparseTensor返回。

## 应用

1. 应用到wide部分：进行embedding_lookup，查找单个id对应的weights，相乘作为该featureField的weighted_sum；
2. 应用到deep部分：进行embedding_lookip，查找单个id对应的emb，聚合后作为该特征域的emb返回。

## 线下训练与线上serving
问题：离线训练shuffle，全部用户，样本；线上serving，单个用户请求处理，batch_size为候选集feed_id个数。